In [1]:
import pandas as pd
import time
import ray
from oanda_forex_utilities import *
import os
import warnings


In [2]:
account_details = get_oanda_account_credentials("demo_account_1.16")
account_id = account_details["account_id"]
access_token = account_details["access_token"]

print(account_id)
print(access_token)

101-003-18483320-019
87a216bb7b0e640e4af97a43499b58b9-4803472d0b66dd35b5d7541f3105caf5


In [3]:

ray.init()
@ray.remote
def automated_trade(currency_pair):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')   
        get_extraction_time("M5")
        time.sleep(1.5)
        state = True
        complete_list = []
        trigger_bar ={}
        while state == True:
            df = get_candle_data(account_id, access_token, currency_pair, "M5", 500)
            df_longer_time = get_candle_data(account_id, access_token, currency_pair, "M30", 500)

            df_with_indicators = get_macd_indicator(df)
            df_with_indicators = get_adx_indicator(df_with_indicators)
            df_longer_time_with_indicators = get_ema_indicator(df_with_indicators, window = 8)
            df_longer_time_with_indicators = get_ema_indicator(df_with_indicators, window = 13)
            df_longer_time_with_indicators = get_ema_indicator(df_with_indicators, window = 21)
            df_with_indicators_complete = df_with_indicators[df_with_indicators["complete"] == True]

            df_longer_time_with_indicators = get_ema_indicator(df_longer_time, window = 8)
            df_longer_time_with_indicators = get_ema_indicator(df_longer_time_with_indicators, window = 13)
            df_longer_time_with_indicators = get_ema_indicator(df_longer_time_with_indicators, window = 21)

            if df_with_indicators_complete["time"].iloc[-1] not in complete_list:
                complete_list.append(df_with_indicators_complete["time"].iloc[-1])
                    if bool(trigger_bar) == False:
                        if (
                                (df_with_indicators_complete["price_high"].iloc[-1] >= df_with_indicators_complete["ema_8"].iloc[-1]) and
                                (df_with_indicators_complete["price_close"].iloc[-1] < df_with_indicators_complete["ema_21"].iloc[-1]) and

                                (df_with_indicators_complete["price_high"].iloc[-2] < df_with_indicators_complete["ema_8"].iloc[-2]) and
                                (df_with_indicators_complete["price_high"].iloc[-3] < df_with_indicators_complete["ema_8"].iloc[-3]) and
                                (df_with_indicators_complete["price_high"].iloc[-4] < df_with_indicators_complete["ema_8"].iloc[-4]) and
        #                         (df_with_indicators_complete["price_high"].iloc[-5] < df_with_indicators_complete["ema_8"].iloc[-5]) and

                                (df_longer_time_with_indicators["price_high"].iloc[-1] < df_longer_time_with_indicators["ema_8"].iloc[-1]) and
                                (df_longer_time_with_indicators["price_high"].iloc[-2] < df_longer_time_with_indicators["ema_8"].iloc[-2])
                            ):
                            print(df_with_indicators_complete["time"].iloc[-1], currency_pair, " SELL TRIGGER BAR")
                            trigger_bar = {"SELL": df_with_indicators_complete["price_close"].iloc[-1]}

                        elif(
                                (df_with_indicators_complete["price_low"].iloc[-1] <= df_with_indicators_complete["ema_8"].iloc[-1]) and
                                (df_with_indicators_complete["price_close"].iloc[-1] > df_with_indicators_complete["ema_21"].iloc[-1]) and

                                (df_with_indicators_complete["price_low"].iloc[-2] > df_with_indicators_complete["ema_8"].iloc[-2]) and
                                (df_with_indicators_complete["price_low"].iloc[-3] > df_with_indicators_complete["ema_8"].iloc[-3]) and
                                (df_with_indicators_complete["price_low"].iloc[-4] > df_with_indicators_complete["ema_8"].iloc[-4]) and
        #                         (df_with_indicators_complete["price_low"].iloc[-5] > df_with_indicators_complete["ema_8"].iloc[-5]) and

                                (df_longer_time_with_indicators["price_low"].iloc[-1] > df_longer_time_with_indicators["ema_8"].iloc[-1]) and
                                (df_longer_time_with_indicators["price_low"].iloc[-2] > df_longer_time_with_indicators["ema_8"].iloc[-2])
                            ):
                            print(df_with_indicators_complete["time"].iloc[-1], currency_pair, " BUY ORDER FILLED")
                            if currency_pair in ["EUR_JPY", "AUD_JPY", "NZD_JPY", "USD_JPY", "GBP_JPY"]:
                                stop_loss = 1000
                            else:
                                stop_loss = 10
                            create_market_order_with_sl_tp(account_id, access_token, currency_pair, "BUY", 5000,
                                                  round(float(df_with_indicators_complete["price_close"].iloc[-1]) - stop_loss/10000,5),
                                                  round(float(df_with_indicators_complete["price_close"].iloc[-1]) + stop_loss/10000,5))

                        else:
                            print(df_with_indicators_complete["time"].iloc[-1], currency_pair, " NOTHING HAPPENS")
                    else:
                        

            time.sleep(10)

2021-08-04 21:05:42,881	INFO services.py:1245 -- View the Ray dashboard at http://127.0.0.1:8267


In [4]:
for i in range(1, 10):
    try:
        ray.get([automated_trade.remote("EUR_USD"), automated_trade.remote("USD_JPY"),
                 automated_trade.remote("USD_CHF"), automated_trade.remote("GBP_USD"),
                 automated_trade.remote("GBP_CHF"), automated_trade.remote("GBP_JPY"),
                 automated_trade.remote("AUD_USD"), automated_trade.remote("EUR_GBP")])
    except Exception as e:
        print (e)
        print ('Restarting!')
        ray.shutdown()
        continue
    else:
        break

(pid=57160) 2021-08-04 13:00:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=57165) 2021-08-04 13:05:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=57161) 2021-08-04 13:05:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=57158) 2021-08-04 13:05:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=57164) 2021-08-04 13:05:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=57162) 2021-08-04 13:05:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=57163) 2021-08-04 13:05:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=57159) 2021-08-04 13:05:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=57160) 2021-08-04 13:05:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=57160) 2021-08-04 13:10:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=57163) 2021-08-04 13:10:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=57158) 2021-08-04 13:10:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=57159) 2021-08-04 13:10:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=57164) 2021-08-04 13:10:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=57165) 2021-08-04 13:10:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=57161) 2021-08-04 13:10:00+00:00 AUD_USD  NOTHING 

(pid=57163) 2021-08-04 14:10:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=57165) 2021-08-04 14:10:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=57162) 2021-08-04 14:10:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=57160) 2021-08-04 14:10:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=57159) 2021-08-04 14:10:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=57160) 2021-08-04 14:15:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=57159) 2021-08-04 14:15:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=57164) 2021-08-04 14:15:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=57161) 2021-08-04 14:15:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=57158) 2021-08-04 14:15:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=57163) 2021-08-04 14:15:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=57165) 2021-08-04 14:15:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=57162) 2021-08-04 14:15:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=57164) 2021-08-04 14:20:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=57159) 2021-08-04 14:20:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=57163) 2021-08-04 14:20:00+00:00 EUR_USD  NOTHING 

(pid=57160) 2021-08-04 15:15:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=57160) 2021-08-04 15:20:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=57159) 2021-08-04 15:20:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=57162) 2021-08-04 15:20:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=57161) 2021-08-04 15:20:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=57158) 2021-08-04 15:20:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=57164) 2021-08-04 15:20:00+00:00 USD_JPY  BUY ORDER FILLED
(pid=57163) 2021-08-04 15:20:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=57165) 2021-08-04 15:20:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=57160) 2021-08-04 15:25:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=57159) 2021-08-04 15:25:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=57162) 2021-08-04 15:25:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=57161) 2021-08-04 15:25:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=57158) 2021-08-04 15:25:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=57163) 2021-08-04 15:25:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=57164) 2021-08-04 15:25:00+00:00 USD_JPY  NOTHING

(pid=57165) 2021-08-04 16:25:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=57163) 2021-08-04 16:25:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=57159) 2021-08-04 16:25:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=57160) 2021-08-04 16:25:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=57162) 2021-08-04 16:25:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=57161) 2021-08-04 16:30:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=57163) 2021-08-04 16:30:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=57158) 2021-08-04 16:30:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=57165) 2021-08-04 16:30:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=57159) 2021-08-04 16:30:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=57160) 2021-08-04 16:30:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=57164) 2021-08-04 16:30:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=57162) 2021-08-04 16:30:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=57165) 2021-08-04 16:35:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=57161) 2021-08-04 16:35:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=57158) 2021-08-04 16:35:00+00:00 GBP_JPY  NOTHING 

(pid=57163) 2021-08-04 17:35:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=57160) 2021-08-04 17:35:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=57158) 2021-08-04 17:35:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=57165) 2021-08-04 17:35:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=57161) 2021-08-04 17:35:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=57164) 2021-08-04 17:35:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=57162) 2021-08-04 17:35:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=57159) 2021-08-04 17:35:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=57159) 2021-08-04 17:40:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=57158) 2021-08-04 17:40:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=57163) 2021-08-04 17:40:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=57160) 2021-08-04 17:40:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=57161) 2021-08-04 17:40:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=57165) 2021-08-04 17:40:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=57164) 2021-08-04 17:40:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=57162) 2021-08-04 17:40:00+00:00 USD_CHF  NOTHING 

...

(pid=57963) 2021-08-04 22:25:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=57962) 2021-08-04 22:25:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=57963) 2021-08-04 22:30:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=57964) 2021-08-04 22:30:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=57958) 2021-08-04 22:30:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=57961) 2021-08-04 22:30:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=57934) 2021-08-04 22:30:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=57960) 2021-08-04 22:30:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=57965) 2021-08-04 22:30:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=57962) 2021-08-04 22:30:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=57964) 2021-08-04 22:35:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=57958) 2021-08-04 22:35:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=57934) 2021-08-04 22:35:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=57961) 2021-08-04 22:35:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=57960) 2021-08-04 22:35:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=57965) 2021-08-04 22:35:00+00:00 EUR_GBP  BUY ORDE

(pid=57960) 2021-08-04 23:40:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=57965) 2021-08-04 23:40:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=57963) 2021-08-04 23:40:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=57962) 2021-08-04 23:40:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=57958) 2021-08-04 23:40:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=57964) 2021-08-04 23:40:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=57961) 2021-08-04 23:40:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=57934) 2021-08-04 23:40:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=57958) 2021-08-04 23:45:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=57963) 2021-08-04 23:45:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=57962) 2021-08-04 23:45:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=57934) 2021-08-04 23:45:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=57964) 2021-08-04 23:45:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=57961) 2021-08-04 23:45:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=57960) 2021-08-04 23:45:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=57965) 2021-08-04 23:45:00+00:00 EUR_GBP  NOTHING 

(pid=57963) 2021-08-05 00:45:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=57960) 2021-08-05 00:45:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=57961) 2021-08-05 00:45:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=57960) 2021-08-05 00:50:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=57961) 2021-08-05 00:50:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=57958) 2021-08-05 00:50:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=57965) 2021-08-05 00:50:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=57962) 2021-08-05 00:50:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=57964) 2021-08-05 00:50:00+00:00 USD_CHF  BUY ORDER FILLED
(pid=57934) 2021-08-05 00:50:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=57963) 2021-08-05 00:50:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=57962) 2021-08-05 00:55:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=57958) 2021-08-05 00:55:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=57965) 2021-08-05 00:55:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=57934) 2021-08-05 00:55:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=57964) 2021-08-05 00:55:00+00:00 USD_CHF  NOTHING

(pid=57934) 2021-08-05 01:55:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=57961) 2021-08-05 01:55:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=57965) 2021-08-05 01:55:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=57958) 2021-08-05 01:55:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=57962) 2021-08-05 01:55:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=57964) 2021-08-05 01:55:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=57963) 2021-08-05 01:55:00+00:00 USD_JPY  NOTHING HAPPENS


KeyboardInterrupt: 

In [5]:
ray.shutdown()

In [7]:
test = {"dash": 1.23}
bool(test)

True